# Análisis Exploratorio de Datos (EDA) - Fashion MNIST

Este notebook realiza un análisis detallado del conjunto de datos **Fashion-MNIST**, analizando su estructura, distribución de clases y características visuales.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Configuración de estilo para seaborn
sns.set(style='whitegrid')

## 1. Carga del Conjunto de Datos

Fashion-MNIST es un conjunto de datos de imágenes de artículos de Zalando, que consiste en un conjunto de entrenamiento de 60,000 ejemplos y un conjunto de prueba de 10,000 ejemplos. Cada ejemplo es una imagen en escala de grises de 28x28, asociada con una etiqueta de 10 clases.

In [ ]:
# Cargar datos
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(f"Dimensiones de entrenamiento: {x_train.shape}")
print(f"Dimensiones de prueba: {x_test.shape}")

# Definir nombres de las clases
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 2. Visualización de Muestras

Visualizamos las primeras 25 imágenes del conjunto de entrenamiento para entender qué tipo de datos estamos manejando.

In [ ]:
plt.figure(figsize=(12,12))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap='gray')
    plt.xlabel(class_names[y_train[i]], fontsize=12)
plt.tight_layout()
plt.show()

## 3. Distribución de Clases

Es crucial verificar si el dataset está balanceado para evitar sesgos en el modelo posterior.

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.countplot(x=y_train, palette='viridis')
plt.xticks(ticks=np.arange(10), labels=class_names, rotation=45)
plt.title('Distribución de Clases en el Conjunto de Entrenamiento', fontsize=15)
plt.xlabel('Clase', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)

# Añadir etiquetas de conteo sobre las barras
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

plt.show()

## 4. Análisis de Intensidad de Píxeles

Analizamos los valores de los píxeles (que van de 0 a 255) para comprender la distribución de luminosidad.

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(x_train.flatten(), bins=50, color='skyblue', edgecolor='black')
plt.title('Distribución de la Intensidad de los Píxeles', fontsize=15)
plt.xlabel('Valor del Píxel', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.show()

## 5. Visualización Promedio por Clase

Podemos visualizar el "promedio" de cada prenda para entender las características comunes.

In [ ]:
plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    class_images = x_train[y_train == i]
    avg_image = np.mean(class_images, axis=0)
    plt.imshow(avg_image, cmap='magma')
    plt.title(class_names[i])
    plt.axis('off')
plt.suptitle('Imagen Promedio por Clase', fontsize=16)
plt.tight_layout()
plt.show()

## 6. Análisis de Variabilidad (Desviación Típica)

La desviación típica nos muestra qué tan diferentes son las prendas dentro de una misma categoría. Las zonas más brillantes indican mayor variabilidad.

In [ ]:
plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    class_images = x_train[y_train == i]
    std_image = np.std(class_images, axis=0)
    plt.imshow(std_image, cmap='viridis')
    plt.title(class_names[i])
    plt.axis('off')
plt.suptitle('Variabilidad (Std Dev) por Clase', fontsize=16)
plt.tight_layout()
plt.show()

## 7. Similitud entre Clases

Calculamos la correlación entre las imágenes promedio de cada clase para identificar cuáles son más propensas a confundirse.

In [ ]:
avg_images = []
for i in range(10):
    avg_images.append(np.mean(x_train[y_train == i], axis=0).flatten())

similarity_matrix = np.corrcoef(avg_images)

plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Matriz de Similitud (Correlación) entre Clases', fontsize=15)
plt.show()

## 8. Análisis de Ocupación de Píxeles (Sparsity)

¿Cuánta área ocupa cada prenda en promedio? Esto puede ser un rasgo distintivo rápido.

In [ ]:
sparsity = []
for i in range(10):
    class_images = x_train[y_train == i]
    # Proporción de píxeles con valor > 10 (considerados no-fondo)
    occupancy = np.mean(class_images > 10)
    sparsity.append(occupancy)

plt.figure(figsize=(12, 6))
sns.barplot(x=class_names, y=sparsity, palette='magma')
plt.title('Porcentaje de Ocupación de la Prenda en la Imagen', fontsize=15)
plt.ylabel('Proporción de Píxeles Activos')
plt.xticks(rotation=45)
plt.show()

## 9. Reducción de Dimensionalidad (PCA y t-SNE)

Utilizamos técnicas de reducción de dimensionalidad para visualizar cómo se agrupan las imágenes en un espacio 2D.

In [ ]:
# Preparar los datos (aplanar y normalizar)
n_samples = 2000 # Usamos un subconjunto para t-SNE por velocidad
x_subset = x_train[:n_samples].reshape(n_samples, -1) / 255.0
y_subset = y_train[:n_samples]

# PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(x_subset)

# t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
tsne_results = tsne.fit_transform(x_subset)

# Visualización
plt.figure(figsize=(16, 7))

plt.subplot(1, 2, 1)
sns.scatterplot(x=pca_results[:,0], y=pca_results[:,1], hue=y_subset, 
                palette='tab10', legend='full', alpha=0.6)
plt.title('Visualización PCA (2D)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')

plt.subplot(1, 2, 2)
scatter = sns.scatterplot(x=tsne_results[:,0], y=tsne_results[:,1], hue=y_subset, 
                palette='tab10', legend='full', alpha=0.6)
plt.title('Visualización t-SNE (2D)')
plt.xlabel('Dimensión t-SNE 1')
plt.ylabel('Dimensión t-SNE 2')

plt.suptitle('Reducción de Dimensionalidad para Fashion-MNIST', fontsize=16)
plt.tight_layout()
plt.show()

## 10. Conclusiones Finales

1. **Balanceo:** El dataset está perfectamente balanceado con 6,000 imágenes por clase.
2. **Similitud Crítica:** La matriz de correlación muestra que 'Shirt', 'T-shirt/top' y 'Pullover' tienen una correlación extremadamente alta (>0.90), lo que confirma que serán el principal reto para el modelo.
3. **Variabilidad:** Las botas (Ankle boot) presentan alta variabilidad en la zona del tobillo, lo que sugiere que hay diferentes alturas de bota en el dataset.
4. **Ocupación:** Los vestidos (Dress) y abrigos (Coat) son las prendas que más área ocupan, mientras que las sandalias (Sandal) son las que menos.
5. **Separabilidad Visual (Reducción de dim.):** En el t-SNE se observa que categorías como 'Trouser' y 'Bag' forman clusters muy definidos y separados, mientras que las prendas de torso ('Shirt', 'Coat', 'Pullover', 'T-shirt') aparecen muy solapadas en el centro del espacio latente.
6. **Estrategia Recomendada:** Dado el solapamiento visual entre prendas superiores, una arquitectura de Red Convolucional (CNN) será necesaria para capturar texturas y detalles finos más allá de la silueta general.